<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#데이터-로드-및-결측치-제거" data-toc-modified-id="데이터-로드-및-결측치-제거-1">데이터 로드 및 결측치 제거</a></span></li><li><span><a href="#위도,-경도-변환할-도로명-주소-리스트-생성" data-toc-modified-id="위도,-경도-변환할-도로명-주소-리스트-생성-2">위도, 경도 변환할 도로명 주소 리스트 생성</a></span></li><li><span><a href="#카카오-API-활용---위도,-경도-변환" data-toc-modified-id="카카오-API-활용---위도,-경도-변환-3">카카오 API 활용 - 위도, 경도 변환</a></span></li></ul></div>

In [23]:
import requests
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import geopandas

import warnings
warnings.filterwarnings(action='ignore')

## 데이터 로드 및 결측치 제거

In [2]:
data = pd.read_csv('data/단독다가구(전월세).csv')
data.head()

,시군구,번지,도로조건,계약면적(㎡),전월세구분,계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명
0,서울특별시 강남구 개포동,1***,-,10.0,전세,201501,16,"9,500",0,1989.0,NaN
1,서울특별시 강남구 개포동,1***,-,15.0,월세,201504,4,200,35,1988.0,논현로6길
2,서울특별시 강남구 개포동,1***,-,15.0,월세,201506,9,500,30,1998.0,논현로18길
3,서울특별시 강남구 개포동,1***,-,15.0,월세,201507,27,500,40,1998.0,개포로20길
4,서울특별시 강남구 개포동,1**,8m미만,15.0,전세,201509,22,"3,000",0,1987.0,선릉로12길


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 762504 entries, 0 to 762503
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   시군구      762504 non-null  object 
 1   번지       762421 non-null  object 
 2   도로조건     762448 non-null  object 
 3   계약면적(㎡)  762504 non-null  float64
 4   전월세구분    762504 non-null  object 
 5   계약년월     762504 non-null  int64  
 6   계약일      762504 non-null  int64  
 7   보증금(만원)  762504 non-null  object 
 8   월세(만원)   762504 non-null  object 
 9   건축년도     726086 non-null  float64
 10  도로명      688663 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 64.0+ MB


In [6]:
data['도로명'].isnull().sum()

73841

In [11]:
data = data.dropna(subset=['도로명']).reset_index(drop=True)
data.head()

,시군구,번지,도로조건,계약면적(㎡),전월세구분,계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명
0,서울특별시 강남구 개포동,1***,-,15.0,월세,201504,4,200,35,1988.0,논현로6길
1,서울특별시 강남구 개포동,1***,-,15.0,월세,201506,9,500,30,1998.0,논현로18길
2,서울특별시 강남구 개포동,1***,-,15.0,월세,201507,27,500,40,1998.0,개포로20길
3,서울특별시 강남구 개포동,1**,8m미만,15.0,전세,201509,22,"3,000",0,1987.0,선릉로12길
4,서울특별시 강남구 개포동,1**,8m미만,15.0,전세,201511,25,"3,000",0,1987.0,선릉로12길


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688663 entries, 0 to 688662
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   시군구      688663 non-null  object 
 1   번지       688663 non-null  object 
 2   도로조건     688607 non-null  object 
 3   계약면적(㎡)  688663 non-null  float64
 4   전월세구분    688663 non-null  object 
 5   계약년월     688663 non-null  int64  
 6   계약일      688663 non-null  int64  
 7   보증금(만원)  688663 non-null  object 
 8   월세(만원)   688663 non-null  object 
 9   건축년도     657947 non-null  float64
 10  도로명      688663 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 57.8+ MB


## 위도, 경도 변환할 도로명 주소 리스트 생성

In [13]:
len(data['도로명'].unique())

12342

In [14]:
add_list = data['도로명'].unique().tolist()
add_list

['논현로6길',
 '논현로18길',
 '개포로20길',
 '선릉로12길',
 '논현로8길',
 '선릉로14길',
 '개포로15길',
 '논현로16길',
 '논현로4길',
 '논현로',
 '개포로17길',
 '논현로12길',
 '논현로10길',
 '논현로2길',
 '논현로20길',
 '개포로28길',
 '개포로24길',
 '개포로32길',
 '개포로22길',
 '개포로25길',
 '선릉로10길',
 '개포로36길',
 '개포로21길',
 '개포로',
 '언주로6길',
 '개포로31길',
 '선릉로8길',
 '논현로14길',
 '개포로34길',
 '선릉로16길',
 '선릉로18길',
 '개포로30길',
 '개포로26길',
 '선릉로6길',
 '강남대로118길',
 '봉은사로25길',
 '논현로124길',
 '봉은사로33길',
 '봉은사로11길',
 '논현로122길',
 '강남대로132길',
 '언주로122길',
 '강남대로126길',
 '학동로4길',
 '논현로136길',
 '선릉로125길',
 '학동로12길',
 '도산대로54길',
 '학동로2길',
 '강남대로120길',
 '강남대로150길',
 '선릉로137길',
 '강남대로122길',
 '봉은사로51길',
 '선릉로115길',
 '학동로42길',
 '선릉로129길',
 '학동로43길',
 '언주로149길',
 '학동로47길',
 '학동로18길',
 '봉은사로1길',
 '봉은사로7길',
 '언주로113길',
 '논현로126길',
 '강남대로128길',
 '강남대로116길',
 '강남대로124길',
 '학동로6길',
 '봉은사로29길',
 '강남대로114길',
 '언주로150길',
 '논현로142길',
 '선릉로141길',
 '논현로114길',
 '언주로147길',
 '학동로48길',
 '학동로41길',
 '선릉로127길',
 '논현로128길',
 '도산대로12길',
 '학동로30길',
 '논현로149길',
 '강남대로146길',
 '도산대로4길',
 '도산대로30길',
 '학동로24길',
 '논현로119길'

## 카카오 API 활용 - 위도, 경도 변환

In [24]:
# 주소 검색 함수 정의
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK *****"})
    json_obj = result.json()
    global val   # 전역 변수를 지역 범위에서 사용하고 싶을 경우
    for document in json_obj['documents']:
        # 값이 NoneType 일 경우의 오류 방지
        try:
            val = [document['address_name'], document['y'], document['x']]
        except:
            val = [0, 0, 0]
    return val

In [42]:
# 주소 검색 함수 적용
address_to_latlon('온수역')

['서울 구로구 온수동', '37.4941466665876', '126.820586087352']

In [18]:
# 주소 검색 함수 반복 적용
list1 = []
for i in add_list:
    list1.append(address_to_latlon(i))

In [19]:
data2 = pd.DataFrame(add_list, columns=['도로명'])
data2.head()

,도로명
0,논현로6길
1,논현로18길
2,개포로20길
3,선릉로12길
4,논현로8길


In [20]:
data3 = pd.DataFrame(list1, columns=['주소','위도','경도']).drop('주소', axis=1)
data3.head()

,위도,경도
0,37.4735609971211,127.050895232485
1,37.4778848357129,127.046338152861
2,37.477039601042,127.048356001948
3,37.4838993527963,127.060991290466
4,37.4744673496263,127.051458050333


In [21]:
data4 = pd.concat([data2, data3], axis=1)
data4.head()

,도로명,위도,경도
0,논현로6길,37.4735609971211,127.050895232485
1,논현로18길,37.4778848357129,127.046338152861
2,개포로20길,37.477039601042,127.048356001948
3,선릉로12길,37.4838993527963,127.060991290466
4,논현로8길,37.4744673496263,127.051458050333


In [22]:
data4.to_csv('data/단독다가구_위도경도.csv', index=False)